#Read bronze table

In [0]:
df = spark.table("workspace.bronze.epr_loc")
df.display()

#Transformation

##remove unwanted space

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, trim,StringType,regexp_replace

In [0]:

for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))
        df.display()

##clean id

In [0]:

df = df.withColumn("cid", F.regexp_replace(col("cid"), "-", ""))

##normalization

In [0]:
df = df.withColumn(
    "cntry",
    F.when(col("cntry") == "DE", "Germany")
     .when(col("cntry").isin("US", "USA"), "United States")
     .when((col("cntry") == "") | col("cntry").isNull(), "n/a")
     .otherwise(col("cntry"))
)

##rename column

In [0]:
RENAME = {
    "cid": "customer_number",
    "cntry": "country"
}
for old, new in RENAME.items():
    df = df.withColumnRenamed(old, new)
     

In [0]:
df.limit(10).display()

#Writing 

In [0]:
df.write.mode("overwrite").saveAsTable("Workspace.silver.epr_loc")

##checks table


In [0]:
%sql
select*from workspace.silver.epr_loc